In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict
import math

In [3]:
train_df = pd.read_csv("~/Thesis/Data/train.csv")
test_df = pd.read_csv("~/Thesis/Data/test.csv")
i2u = defaultdict(list)
for data in train_df.values:
    i2u[data[1]].append(data[0])

In [4]:
def ndcg(k, ranklist, testlist):
    if not testlist: return 0
    idcg_k, dcg_k = 0, 0
    if len(testlist) < k:
        k = len(testlist)
    for i in range(k):
        idcg_k += 1 / math.log(i + 2, 2)
        
    s = set(testlist)
    hits = [idx for idx, val in enumerate(ranklist) if val in s]
    count = len(hits)
    
    for i in range(count):
        dcg_k += 1 / math.log(hits[i] + 2, 2)
        
    return float(dcg_k / idcg_k)

In [5]:
test_users, simi, precision, final = set(test_df.user), {}, [], []
recall, f1, ndcg_k = [], [], []
for k in range(9, 10):
    ndcg_k = []
    precision, simi = [], {}
    for user in test_users:
        simi = {}
#         animes = test_df[test_df.user == user]
#         items = animes[animes.rating == 10].anime_id.tolist()
#         if items: 
#             item = items[0]
#         else: 
        target_items = test_df[(test_df.user == user)].anime.tolist()
        for i in range(10, -1, -1):
            items = test_df[(test_df.rating == i) & (test_df.user == user)].anime.tolist()
            if items:
                item = items[0]
                break
                    
        for key in target_items:
            if not i2u[item] or not i2u[key]:
                simi[(item, key)] = 0
                continue
            simi[(item, key)] = len(set(i2u[item]) & set(i2u[key])) / math.sqrt(len(i2u[item]) * len(i2u[key]))

        rec = [item[0][1] for item in sorted(simi.items(), key = lambda x: x[1], reverse = True)][1: 6]
        
        temp = test_df[(test_df.rating > k) & (test_df.user == user)].anime.tolist()
#         both = set(temp) & set(rec)
#         precise = len(both) / len(rec)
        
#         precision.append(len(both) / len(rec))
#         recall.append(len(both) / len(temp))
        ndcg_k.append(ndcg(10, rec, temp))
    
    final.append(np.mean(ndcg_k))
    print(final[-1])

0.2364278845188507


In [11]:
print(final)

[1.0, 0.9962561157200595, 0.9911933631142311, 0.9838332269729845, 0.9675388215273345, 0.9372899383109978, 0.8802807913209957, 0.7497553711976175, 0.5318868325888109, 0.277898319506488]


In [17]:
m, similarity, i2u = len(item_to_user), {}, list(item_to_user.items())
for i in range(m):
    if not i % 100: print(i)
    for j in range(i, m):
        similarity[(i2u[i][0], i2u[j][0])] = len(set(i2u[i][1]) & set(i2u[j][1])) / math.sqrt(len(i2u[i][1]) * len(i2u[j][1]))

print(len(similarity))

0
100


KeyboardInterrupt: 